<a href="https://colab.research.google.com/github/LeonardoLiberty/algo/blob/master/Bert_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [13]:
import os 
os.chdir('/content') # switch the dictionary to content
os.getcwd()

'/content'

In [14]:
train = pd.read_csv('common-sense-reason-challenge/train_data.csv')  
train.head()

,id,FalseSent,OptionA,OptionB,OptionC
0,sentence_1,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings
1,sentence_2,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.
2,sentence_3,A normal closet is larger than a walk-in closet.,Walk-in closets are normal closets.,A person can sleep in a walk-in closet.,A person cannot walk into a normal closet beca...
3,sentence_4,I like to ride my chocolate,Chocolate is delicious and bikes are not,"Chocolate is a food, not a transportation unit",My bike can't ride a chocolate
4,sentence_5,A GIRL WON THE RACE WITH HORSE,GIRL HAVE BEAUTIFUL HAIR BUT THE HORSE DOESN'T...,THE GIRL WEAR DRESS BUT THE HORSE DOESN'T HAVE .,HORSE RAN FASTER THAN HER


In [15]:
train_ans = pd.read_csv('common-sense-reason-challenge/train_answers.csv')  
train_ans.head(10)

,id,answer
0,sentence_1,C
1,sentence_2,B
2,sentence_3,C
3,sentence_4,B
4,sentence_5,C
5,sentence_6,B
6,sentence_7,A
7,sentence_8,B
8,sentence_9,C
9,sentence_10,B


In [18]:
# !pip install transformers

     |████████████████████████████████| 1.4MB 5.5MB/s 
     |████████████████████████████████| 890kB 14.8MB/s 
     |████████████████████████████████| 2.9MB 20.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=68a9f2674e9aa04a3ae7614323826c45e33103cc2b1037ff5ad960a18dbd23f6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [19]:
import torch
from transformers import pipeline
nlp = pipeline("question-answering")

In [22]:
def predict(data)->list:
    split_contains = 0 # Count the split char to check the impact--  *!@#$%^
    predict_result = [] # Return the result option A,B,C
    predict_score = [] # Return the score of select option
    cur = 0 
    step = [0.05*(i+1) for i in range(20)] # count the progress, please ignore
    ans = ['A','B','C']
    for i in range(data.shape[0]):
        q = str(data.iloc[i, 1]) # column Falsesent
        oa, ob, oc = str(data.iloc[i,2]), str(data.iloc[i,3]), str(data.iloc[i,4])
        a = oa + ' *!@#$%^ '+ ob + ' *!@#$%^ '+ oc # joint options
        options = [len(oa), len(oa)+ 9 + len(ob), len(a)] # split option location
        result = nlp(question=q, context=a) 
        # print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}")
        predict_score.append(round(result['score'], 4)) 
        if '*!@#$%^' in result['answer']:
            split_contains += 1
            
        if i/data.shape[0] >= step[cur]:
            print(round(i/data.shape[0], 3))
            cur += 1
        
        for i in range(len(ans)):
            if result['start'] < options[i]:
                predict_result.append(ans[i])
                #print(ans[i])
                break
            
    return predict_result, predict_score, split_contains


predict_result, predict_score, split_contains = predict(train)        
  


0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95


In [23]:
test = pd.read_csv('common-sense-reason-challenge/test_data.csv')  
test.head()


,id,FalseSent,OptionA,OptionB,OptionC
0,sentence_1,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...
1,sentence_2,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human
2,sentence_3,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km."
3,sentence_4,Mark drank a notebook.,The notebook was yellow and Mark does not drin...,A notebook is a solid and by definition a pers...,Notebooks cost $.50
4,sentence_5,He went to the police station to withdraw cash.,Police catches thieves while money can buy you...,Police station is a place to file criminal com...,Police station has jails but no money to give.


In [24]:
train_ans = pd.read_csv('common-sense-reason-challenge/train_answers.csv')  
train_ans.head(10)

,id,answer
0,sentence_1,C
1,sentence_2,B
2,sentence_3,C
3,sentence_4,B
4,sentence_5,C
5,sentence_6,B
6,sentence_7,A
7,sentence_8,B
8,sentence_9,C
9,sentence_10,B


In [26]:
predict_result_test, predict_score_test, split_contains_test = predict(test) 

0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.601
0.65
0.701
0.75
0.8
0.851
0.9
0.951


In [27]:
def evaluation(pred, test):
    # tp, tn, fp, fn = 0, 0, 0, 0
    t = 0
    for i in range(len(pred)):
        if pred[i] == test.iloc[i,1]:
            t += 1

    print(t/len(pred))

evaluation(predict_result, train_ans)
print(split_contains/len(predict_result))

0.48575
0.006375


In [34]:
def create_test_result_file(pred):
  test_ans = (train_ans.copy())[:2000]
  print(type(test_ans))
  print(test_ans.shape)
  test_ans['answer'] = np.array(pred)
  print(test_ans.head())
  return test_ans


test_ans = create_test_result_file(predict_result_test)

<class 'pandas.core.frame.DataFrame'>
(2000, 2)
           id answer
0  sentence_1      B
1  sentence_2      C
2  sentence_3      C
3  sentence_4      A
4  sentence_5      C


In [36]:
test_ans.to_csv(path_or_buf='./test.csv' ,index=False)